In [60]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
from matplotlib import pyplot as plt
%matplotlib inline

from keras.callbacks import EarlyStopping
from keras.layers import *
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras import regularizers, optimizers
import keras.backend as K

In [61]:
train = pd.read_csv("C:/Users/piyushkirtivardhan/Desktop/New folder/train_2.csv-20200824T035958Z-001/train_2.csv/Train_2.csv").fillna(0)

In [62]:
test= pd.read_csv('key_2.csv')
solution=pd.read_csv('sample_submission_2.csv')

In [63]:
train_page = pd.DataFrame([page.split('_')[-3:]for page in tqdm(train.Page)])

100%|██████████████████████████████████████████████████████████████████████| 145063/145063 [00:00<00:00, 310157.91it/s]


In [64]:
for column in train_page.columns:
    train_page['c_%d' % (column)] = train_page[column].astype('category').cat.codes

In [65]:
#converting the value to categorical 
language = to_categorical(train_page.iloc[:, -3])
access_type = to_categorical(train_page.iloc[:, -2])
agent_type = to_categorical(train_page.iloc[:, -1])

In [66]:
date = train.iloc[:, 1:].astype('float32')

In [67]:
#Transposing the df coloums 
date = date.T.fillna(date.median(axis=1)).T

In [68]:
#converting the values in the log
date = np.log1p(date).values

In [69]:
start_date = datetime.datetime(2017, 9, 13)
end_date= datetime.datetime(2017, 11, 13)
step = datetime.timedelta(days=1)

result = []

while start_date <= end_date:
    result.append(start_date.strftime('%Y-%m-%d'))
    start_date += step

In [70]:
train_index = range((802-365*2+1+239+2), (802-365+1))
pred_index = range((802-365+1+239+2), 803)
y_test=range(439,563)

y_valid = date[:, (802-365+1+2):(802-365+1+64)]

In [71]:
def k_smape(y_true, y_pred):
    '''Symmetric mean absolute percentage error for keras metric'''
    diff = K.abs(y_true - y_pred) / \
        K.clip((K.abs(y_true) + K.abs(y_pred))/2, K.epsilon(), None)
    return K.mean(diff, axis=-1)

In [72]:
def function_1(data_point, best_model):
    '''This function will take features and predict the label using best model'''
    label_1 = best_model.predict(data_point)
    y_pred = np.expm1(label_1)
    output = pd.DataFrame(y_pred, columns=result)
    output['Page'] = train.Page
    output = output.melt(id_vars='Page')
    output.Page = output.Page+'_'+output.variable
    final = test.merge(output, how='left')[['Id', 'value']].rename(columns={'value': 'Visits'})
    print(final)
    
    

In [73]:
from tensorflow.keras.models import load_model
best_model = load_model('model.h5',custom_objects={"k_smape": k_smape})

In [74]:
# this function will take the x_test data points and predict the visits of sites
function_1([date[:, pred_index],language, access_type, agent_type], best_model)

                   Id      Visits
0        0b293039387a  506.505463
1        7114389dd824  502.362030
2        057b02ff1f09  511.059570
3        bd2aca21caa3  504.015625
4        c0effb42cdd5  509.990753
...               ...         ...
8993901  f78fbaa305ad  194.686813
8993902  33aecaf259d1  211.532135
8993903  a36228b64466  227.865845
8993904  26887d3e5c99  266.522797
8993905  06ad06979f80  275.555298

[8993906 rows x 2 columns]


In [75]:
def function_2(data_point, label, best_model):
    label_1 = best_model.predict(data_point)
    y_pred = np.expm1(label_1)
    output = pd.DataFrame(y_pred, columns=result)
    output['Page'] = train.Page
    output = output.melt(id_vars='Page')
    output.Page = output.Page+'_'+output.variable
    final = test.merge(output, how='left')[['Id', 'value']].rename(columns={'value': 'Visits'})
    a = final.Visits
    b = label
    c=np.mean(np.abs(a-b)/np.clip((np.abs(a)+np.abs(b))/2, 1e-7, None))
    print(c)


In [76]:
function_2(([date[:, pred_index],language, access_type, agent_type]),solution.Visits ,best_model)

2.0


In [77]:
train_1=date[:, pred_index]

In [106]:
def function_3(data_point, best_model):
    '''This function will take features and predict the label using best model'''
    label_1 = best_model.predict(data_point)
    y_pred = np.expm1(label_1)
    output = pd.DataFrame(y_pred, columns=result)
    print(output)

In [108]:
function_3([train_1[:1],language[:1], access_type[:1], agent_type[:1]], best_model)

   2017-09-13  2017-09-14  2017-09-15  2017-09-16  2017-09-17  2017-09-18  \
0   25.598751   26.247641   25.606812   27.235538   29.582291   29.380312   

   2017-09-19  2017-09-20  2017-09-21  2017-09-22  ...  2017-11-04  \
0   28.050175   26.063652   26.193773   26.151852  ...   25.019234   

   2017-11-05  2017-11-06  2017-11-07  2017-11-08  2017-11-09  2017-11-10  \
0   24.482029   25.898239   25.423105    23.61108   20.411545   21.221523   

   2017-11-11  2017-11-12  2017-11-13  
0   19.934231   25.567276   23.946735  

[1 rows x 62 columns]
